# Transformers para Martín Fierro

Los principales objetivos de este tutorial serán mostrar cómo es una arquitectura *transformer* y cómo usar las herramientas provistas por [Hugging Face](https://huggingface.co/). En este tutorial usaremos una implementación de [GPT2](https://openai.com/blog/better-language-models/) en español para generar (una vez más) texto similar al **Martín Fierro**, que se puede descargar [aquí](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt).

Créditos a [Jay Alammar](https://jalammar.github.io/illustrated-gpt2/) por las imágenes.

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 28.2 MB/s 
     |████████████████████████████████| 596 kB 30.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import pandas as pd

import torch

import transformers

use_cuda = torch.cuda.is_available()

## GPT2

**GPT2** es un modelo de generación de texto basado en *transformers*. A diferencia de **BERT** u otros transformadores, **GPT2** está basado en sucesivas instancias de *decoders*.

<img src='images/7_Comparison.png'
     alt='GPT2 Decoder Architecture'
     style='float: center; margin-right: 150px;'
     width=75%
     />

<img src='images/7_Function.gif'
     alt='GPT2 Text Generation Function'
     style='float: center; margin-right: 150px;'
     width=75%
     />

**Recursos:**

[The Illustrated GPT-2 (Visualizing Transformer Language Models)](https://jalammar.github.io/illustrated-gpt2/)

## Parte 1: Usando transformers de Hugging Face

La librería *transformers* permite usar modelos ya entrenados en un pipeline y nos facilita funciones que procesan el input, lo tokenizan, y hace el forward pass para generar las predicciones.

Dada una tarea, pipeline descarga un modelo y un tokenizador apropiados para la misma. En este caso, especificamos un modelo preentrenado de *GPT2-small* en idioma español.

<img src='images/7_Sizes.png'
     alt='GPT2 Model Sizes'
     style='float: center; margin-right: 200px;'
     width=70%
     />

<img src='images/7_Stacks.png'
     alt='GPT2 Decoder Stacks'
     style='float: center; margin-right: 200px;'
     width=70%
     />

Para nuestra tarea en particular, pipeline tomará un texto e internamente se encargará del tokenizado de la oración y de generar texto hasta el *max_length* indicado.

In [3]:
from transformers import pipeline

generator = pipeline('text-generation', model='datificate/gpt2-small-spanish')

Downloading:   0%|          | 0.00/817 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/620 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/830k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [4]:
input_sentence = 'Yo he visto en esa milonga muchos Gefes con estancia,'
output = generator(input_sentence, max_length=50)

print('Generated Text...')
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text...
Yo he visto en esa milonga muchos Gefes con estancia, y el amor más profundo.




Para el libro "The Best Of the Universe of the Sea", el escritor John Morris llama "un álbum de misterio". Por


Para usar otras tareas de 🤗 *Transformers*, [aquí](https://huggingface.co/transformers/task_summary.html) hay una lista completa de todas las tareas que se implementan.

---

## Parte 1: Importando 🤗 Transformers

Lo primero es importar el modelo con el cuál trabajaremos. En nuestro caso usaremos [GPT2-Small en español de Datificate](https://huggingface.co/datificate/gpt2-small-spanish) (pueden jugar con el modelo desde la misma página de 🤗).

Nuestra tarea de interés es hacer *Language Modeling* del **Martín Fierro**, por lo cual importamos **GPT2LMHeadModel**, que es una implementación de GPT2 con la última capa lineal para retornar los *logits* del tamaño del vocabulario.

Como el modelo *datificate/gpt2-small-spanish* fue entrenado en *tensorflow*, usamos el flag *from_tf=True* para asegurarnos que importe los pesos correctamente.

In [5]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('datificate/gpt2-small-spanish', from_tf=True)

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


El paso siguiente es descargar un tokenizador. El tokenizador hará por nosotros el trabajo de separar el texto en palabras y convertirlas en sus correspondientes *ids* dentro del vocabulario. Al igual que con el modelo, 🤗 provee tokenizers ya implementados para sus correspondientes modelos.

In [6]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('datificate/gpt2-small-spanish', add_prefix_space=True)

tokenized_seq = tokenizer('Yo nunca habia escuchado hablar de Innsmouth hasta')

print(f'inputs_ids: {tokenized_seq["input_ids"]}')
print(f'attention_mask: {tokenized_seq["attention_mask"]}')

tokens = tokenizer.convert_ids_to_tokens(tokenized_seq['input_ids'])

print(f'tokens: {tokens}')

inputs_ids: [4701, 2686, 498, 363, 27422, 6242, 258, 599, 10007, 15308, 657]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens: ['ĠYo', 'Ġnunca', 'Ġhab', 'ia', 'Ġescuchado', 'Ġhablar', 'Ġde', 'ĠIn', 'ns', 'mouth', 'Ġhasta']


El tokenizer se puede encargar por nosotros de manejar temas de padding y truncado entre otros. Para más información sobre las funciones del tokenizer, investigar [aquí](https://huggingface.co/transformers/main_classes/tokenizer.html).

El tokenizer también puede procesar múltiples oraciones al mismo tiempo, y retornar tensores para el framework que estemos usando de base, sea este *TensorFlow* (`tf`) o *PyTorch* (`pt`).

In [7]:
sentences = [
    'Yo nunca habia escuchado hablar de Innsmouth hasta',
    'Hacia el oeste de Arkham las montañas se levantaban indómitas y en sus entrañas'
]

In [8]:
tokenized_seq = tokenizer(sentences, padding=True, return_tensors='pt')

print(f'inputs_ids...\n{tokenized_seq["input_ids"]}')
print(f'attention_mask...\n{tokenized_seq["attention_mask"]}')

inputs_ids...
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask...
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Incluso el tokenizer puede manejar listas de palabras con solo agregar el parámetro `is_split_into_words=True`.

In [9]:
tokenized_seq = tokenizer([sent.split() for sent in sentences],
                          padding=True,
                          return_tensors='pt',
                          is_split_into_words=True)

print(f'inputs_ids...\n{tokenized_seq["input_ids"]}')
print(f'attention_mask...\n{tokenized_seq["attention_mask"]}')

inputs_ids...
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask...
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


---

## Parte 2: Finetune con PyTorch nativo

Para entrenar con *PyTorch*, necesitamos crear una instancia de un *Dataset*. Esta vez, podemos usar el tokenizer que ya importamos para hacer el encoding de las palabras.

Por simplicidad, dividiremos el conjunto de datos en palabras y usaremos listas de palabras de tamaño fijo.

Para entrenamiento nativo en *TensorFlow*, ver [aquí](https://huggingface.co/transformers/training.html#fine-tuning-in-native-tensorflow-2).

In [10]:
from torch.utils.data import Dataset

class MartinFierro_Dataset_GPT2(Dataset):
    def __init__(self, text_data, max_len, tokenizer):
        self.max_len = max_len
        # Get ourselves a list of words so we can iterate
        split_text = text_data.split()

        # Cut the text in sequences of max_len characters
        self.sentences = {}
        for idx, i in enumerate(range(0, len(split_text) - max_len - 1, max_len)):
            self.sentences[idx] = split_text[i: i + max_len]

        # You need to activate padding in order to return tensors
        self.data = tokenizer(list(self.sentences.values()),
                              padding=True,
                              is_split_into_words=True,
                              return_tensors='pt')

        self.length = len(self.sentences)
        print(f'NB sequences: {self.length}')

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        return item

Cargamos el archivo e instanciamos el dataset.

In [11]:
import re
import unicodedata

with open('martin_fierro.txt', 'r') as finput:
    text = unicodedata.normalize('NFC', finput.read()).lower()
    text = re.sub('\s+', ' ', text).strip()

print(f'Corpus length: {len(text)}')

train_dataset = MartinFierro_Dataset_GPT2(text, 50, tokenizer)

Corpus length: 33858
NB sequences: 128


Configuramos nuestro modelo en modo *train* y preparamos el dispositivo donde vamos a ejecutar.

In [12]:
model.train()

device = torch.device('cuda') if use_cuda else torch.device('cpu') 
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


Importamos de *transformers* la implementacion de [AdamW](https://arxiv.org/abs/1711.05101).

In [13]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=0.00001, weight_decay=0.01)

Instanciamos nuestro *dataloader* con el dataset del **Martín Fierro**.

In [14]:
from torch.utils.data import DataLoader

dataloader_config = {'dataset': train_dataset,
                     'batch_size': 8,
                     'shuffle': True,
                     'num_workers': 0,
                     'pin_memory': use_cuda}

dataloader = DataLoader(**dataloader_config)

Finalmente hacemos un **forward pass** por todo el dataset (i.e. entrenamos por una *epoch*). El modelo importado de 🤗 *transformers* ya se encarga de calcular la pérdida para la tarea que necesitamos (en este caso es un **LM**) en el *forward pass* si le pasamos el argumento *labels*.

In [15]:
from tqdm.notebook import tqdm

stream = tqdm(enumerate(dataloader), total=len(dataloader))
for i, sample in stream:
    # Send everything to the device!
    input_ids = sample['input_ids'].to(device)
    attention_mask = sample['attention_mask'].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
    optimizer.zero_grad()
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    stream.set_postfix({'loss': loss.detach().cpu().numpy()})

  0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


## Parte 3: Entrenando con *Transformers Trainers*

🤗 también provee sus propios métodos para entrenar. En este caso:
  - *TrainingArgument*: nos genera una configuración para un entrenamiento.
  - *Trainer*: clase que se encargará del entrenamiento por nosotros.
  
Importamos el *Data Collator* necesario para nuestra tarea y pasamos los argumentos necesario para cada objeto.

In [16]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# We're not training with MLM, so we must set mlm=False
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./GPT2_Fierro',      # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=1,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                     # the instantiated Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset (THE SAME AS BEFORE)
    data_collator=data_collator
)

Finalmente, entrenar es tan simple como llamar al método `train()` del entrenador.

In [17]:
trainer.train()

***** Running training *****
  Num examples = 128
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=16, training_loss=5.899720668792725, metrics={'train_runtime': 7.5413, 'train_samples_per_second': 16.973, 'train_steps_per_second': 2.122, 'total_flos': 5683101696000.0, 'train_loss': 5.899720668792725, 'epoch': 1.0})

---

## Parte 4: Usando nuestro nuevo modelo en un pipeline

In [18]:
model.cpu()

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [19]:
input_sentence = 'Yo he visto en esa milonga muchos Gefes con estancia,'
output = generator(input_sentence, max_length=50)

print('Generated Text...')
print(output[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text...
Yo he visto en esa milonga muchos Gefes con estancia, y yo me quí he visto en la puerta, la morena los palomos llenos de panza. me ha hecho a buscar yo el sol que me ha llevado. me
